# Objective: To develop a mathematical understanding of PCA

# Test data

In [1]:
import numpy as np
from scipy import linalg as LA
X = np.array([
        [0.387,4878, 5.42],
        [0.723,12104,5.25],
        [1,12756,5.52],
        [1.524,6787,3.94],
    ])
X = X - np.mean(X, axis=0)
print(X.shape)

(4, 3)


# Non-Linear Iterative Partial Least-Squares (NIPALS) algorithm

Steps to compute PCA using NIPALS algorithm

* Step 1: Initialize an arbitrary column vector $\mathbf{t}_{a}$ either randomly or by just copying any column of X. 
* Step 2: Take every column of $\mathbf{X}$, $\mathbf{X_k}$ and regress it onto the $\mathbf{t}_{a}$ vector and store the regression coefficeints as $\mathbf{p}_{ka}$. (Note: This simply means performing an ordinary least squares regression ($y=mx$) with $x=t_{a}$ and $y=X_{k}$ with $m=(\mathbf{x^T}\mathbf{x})^{-1}\mathbf{x^T}\mathbf{y}$). In the current notation we get 
$$p_{ka}=\frac{\mathbf{t_a^T}\mathbf{X}_{k}}{\mathbf{t_a^T}\mathbf{t_a}}$$

Repeat it for each of the columns of $X$ to get the entire vector $\mathbf{p}_{k}$. This is shown in the illustration
above where each column from $X$ is regressed, one at a time, on $\mathbf{t}_{a}$, to calculate the loading entry, $\mathbf{𝑝}_{ka}$ 

In practice we don’t do this one column at time; we can regress all columns in $X$ in go: $$\mathbf{p_a^T}=\frac{1}{\mathbf{t_a^T}\mathbf{t_a}}.\mathbf{t_a^T}\mathbf{X_a}$$  where $\mathbf{t_a}$ is an $N \times 1$ column vector, and $\mathbf{X}_{a}$ us an $N \times K$ matrix.
* The loading vector $\mathbf{p_a^T}$ won’t have unit length (magnitude) yet. So we simply rescale it to have
magnitude of 1.0: $$\mathbf{p_a^T}=\frac{\mathbf{p_a^T}}{\sqrt{\mathbf{p_a^T}\mathbf{p_a}}}$$
* Step 4: Regress every row in $X$ onto this normalized loadings vector. As illustrated below, in our linear regression the rows in X are our y-variable each time, while the loadings vector is our x-variable. The regression coefficient becomes the score value for that $𝑖^{th}$ row:

$$t_{i,a}=\frac{\mathbf{x}_{i}^{T}\mathbf{p}_{a}}{\mathbf{p}_{a}^{T}\mathbf{p}_{a}}$$
where $x_{i}^{T}$ is a $K \times 1$ column vector. We can combine these $N$ separate least-squares models and
calculate them in one go to get the entire vector, 

$$\mathbf{t}_{a}^{T}=\frac{1}{\mathbf{p}_{a}^{T}\mathbf{p}_{a}}\mathbf{X}\mathbf{p}_{a}^{T}$$  where $p_{a}$ is a $K \times 1$ column vector.
* Step 5: Continue looping over steps 2,3,4 until the change in vector $t_{a}$ is below a chosen tolerance
* Step 6: On convergence, the score vector and the loading vectors, $\mathbf{t}_{a}$ and $\mathbf{p}_{a}$ are stored as the $a^{th}$ column in matrix $\mathbf{T}$ and $\mathbf{P}$. We then deflate the $\mathbf{X}$ matrix. This crucial step removes the variability captured in this component ($t_{a}$ and $p_{a}$) from $\mathbf{X}$:

$$E_{a}=X_{a}-t_{a}p_{a}^{T}$$

$$X_{a+1} = E_{a}$$ 

For the first component, $X_{a}$ is just the preprocessed raw data. So we can see that the second component is actually calculated on the residuals $E_{1}$, obtained after extracting the first component. This is called deflation, and nicely shows why each component is orthogonal to the others. Each subsequent component is only seeing variation remaining after removing all the others; there is no possibility that two components can explain the same type of variability. After deflation we go back to step 1 and repeat the entire process for the next component. 

In short:

1. Guess a scores vector.

2. Find a loadings vector that fits this.

3. Normalize the found loadings.

4. Do regression on all rows of X to find the new scores.

5. Do step 2-4 over and over until change is small enough.

6. The scores and loadings left are the PCs.

## IMPLEMENTATION IN PYTHON

In [2]:
def PCA(X,no_components):
    tol = 0.0000001
    it=1000
    obsCount,varCount = X.shape
    Xa = X - np.mean(X, axis = 0) 
    #Xh = X-np.tile(np.mean(X,axis=0).reshape(-1,1).T, obsCount).reshape(4,3)
    T = np.zeros((obsCount,no_components))
    P = np.zeros((varCount,no_components))
    pcvar = np.zeros((varCount,1))
    varTotal = np.sum(np.var(Xa,axis=0,ddof=1))
    currVar = varTotal
    nr=0
    for h in range(no_components):
        th = Xa[:,0].reshape(-1,1)
        ende = False
        while ende != True:
            nr = nr + 1
            ph = np.dot(Xa.T,th)/np.dot(th.T,th)
            ph = ph /np.linalg.norm(ph)
            thnew = np.dot(Xa,ph)/np.dot(ph.T,ph)
            prec = np.dot((thnew-th).T,(thnew-th))
            th = thnew
            if prec <= (tol*tol):
                ende = True
            elif it <=nr:
                ende = True
                print("Iternation stops without convergence")
        Ea = Xa - np.dot(th,ph.T)
        Xa = Ea    
        T[:,h] = th.flatten()
        P[:,h] = ph.flatten()
        oldVar = currVar
        currVar = np.sum(np.var(Xa,axis=0,ddof=1))
        pcvar[h] = (oldVar - currVar) / varTotal
        nr = 0
    return T,P,pcvar      

## Advantages of the NIPALS algorithm
* The NIPALS algorithm computes one component at a time. The first component computed is
equivalent to the t1 and p1 vectors that would have been found from an eigenvalue or singular value
decomposition.
* The algorithm can handle missing data in X.
* The algorithm always converges, but the convergence can sometimes be slow.
* It is also known as the Power algorithm to calculate eigenvectors and eigenvalues.
* It works well for very large data sets.
* It is used by most software packages, especially those that handle missing data.
* Of interest: it is well known that Google used this algorithm for the early versions of their search engine, called PageRank148.

In [3]:
no_components=3
T,P,pcvar = PCA(X,no_components)
print("T (Scores)")
print(T)
print(" ")
print("P (Loadings)")
print(P)
print(np.sqrt(pcvar)/np.sum(np.sqrt(pcvar)))

T (Scores)
[[-4.25324997e+03 -8.41288672e-01  8.37859036e-03]
 [ 2.97275001e+03 -1.25977272e-01 -1.82476780e-01]
 [ 3.62475003e+03 -1.56843494e-01  1.65224286e-01]
 [-2.34425007e+03  1.12410944e+00  8.87390330e-03]]
 
P (Loadings)
[[ 1.21901390e-05  5.66460728e-01  8.24088735e-01]
 [ 9.99999997e-01  5.32639787e-05 -5.14047689e-05]
 [ 7.30130279e-05 -8.24088733e-01  5.66460726e-01]]
[[9.99753412e-01]
 [2.10083377e-04]
 [3.65048880e-05]]


# SVD

In [4]:
from numpy.linalg import svd 
U, S, PTrans = svd(X, full_matrices=False)
Sigma = np.diag(S)
T=np.dot(U,Sigma)
P=PTrans.T

print("T (Scores)")
print(T)
print(" ")
print("P (Loadings)")
print(P)
print("Sigma (Variance)")
print(S)

T (Scores)
[[-4.25324997e+03 -8.41288672e-01 -8.37858943e-03]
 [ 2.97275001e+03 -1.25977271e-01  1.82476780e-01]
 [ 3.62475003e+03 -1.56843494e-01 -1.65224286e-01]
 [-2.34425007e+03  1.12410944e+00 -8.87390454e-03]]
 
P (Loadings)
[[ 1.21901390e-05  5.66460727e-01 -8.24088736e-01]
 [ 9.99999997e-01  5.32639789e-05  5.14047691e-05]
 [ 7.30130279e-05 -8.24088734e-01 -5.66460725e-01]]
Sigma (Variance)
[6.74994067e+03 1.41840009e+00 2.46466604e-01]


# SKLEARN PCA

In [5]:
from sklearn.decomposition import PCA
pca = PCA()  
T=pca.fit_transform(X)
Prans=pca.components_ #eigen vectors.T
latent = pca.explained_variance_
explained = pca.explained_variance_ratio_
P=PTrans.T
S=pca.singular_values_
Sigma=np.diag(S)
print("T (Scores)")
print(T)
print(" ")
print("P (Loadings)")
print(P)
print("Sigma (Variance)")
print(S)
#print(pca.singular_values_/np.sqrt(3))

T (Scores)
[[ 4.25324997e+03 -8.41288672e-01 -8.37858943e-03]
 [-2.97275001e+03 -1.25977271e-01  1.82476780e-01]
 [-3.62475003e+03 -1.56843494e-01 -1.65224286e-01]
 [ 2.34425007e+03  1.12410944e+00 -8.87390454e-03]]
 
P (Loadings)
[[ 1.21901390e-05  5.66460727e-01 -8.24088736e-01]
 [ 9.99999997e-01  5.32639789e-05  5.14047691e-05]
 [ 7.30130279e-05 -8.24088734e-01 -5.66460725e-01]]
Sigma (Variance)
[6.74994067e+03 1.41840009e+00 2.46466604e-01]


In [6]:
pca.explained_variance_ratio_

array([9.99999955e-01, 4.41567976e-08, 1.33326424e-09])

In [7]:
explained_variance_2 = (S ** 2) / 4
explained_variance_ratio_2 = (explained_variance_2 / explained_variance_2.sum())
print(explained_variance_ratio_2)

[9.99999955e-01 4.41567976e-08 1.33326424e-09]


# Eigenvalue decomposition approach

Recall that the latent variable directions (the loading vectors) were oriented so that the variance of the
scores in that direction were maximal. We can cast this as an optimization problem. For the first
component: $$max (\phi)=\mathbf{t_1^T}\mathbf{t_1}=\mathbf{p_1^T} \mathbf{X^T}\mathbf{Xp_1}$$
subject to $$\mathbf{p_1^T p_1}=1$$.

This is equivalent to $$max (\phi)=\mathbf{p_1^T} \mathbf{X^T Xp_1}-\lambda(\mathbf{p_1^T}\mathbf{p_1}-1)$$ 

because we can move the constraint into the objective function with a Lagrange multiplier, $\lambda$. The maximum value must occur when the partial derivatives with respect to $\mathbf{p_1}$, 

our search variable, are zero: $$\frac{\partial \phi}{\partial p_1}= \frac{\partial (\mathbf{p_1^T X^T Xp_1}-\lambda(\mathbf{p}_{1}^{T}\mathbf{p}_{1}-1))}{\partial \mathbf{p}_1}=0$$

$$2\mathbf{X^T X p_1}-2\lambda_1\mathbf{p_1}=0$$

$$(\mathbf{X^TX}-\lambda_1\mathbf{I})\mathbf{p_1}=0$$

$$\mathbf{X^T Xp_1}=\lambda_{1}\mathbf{p_1}$$

which is just the eigenvalue equation, indicating that $\mathbf{p_1}$ is the eigenvector of $\mathbf{X^T X}$ and $\lambda_1$ is the eigenvalue. One can show that $\lambda_1=\mathbf{t_1^T t_1}$, which is proportional to the variance of the first component. In a similar manner we can calculate the second eigenvalue, but this time we add the additional constraint that $\mathbf{p}_1 \perp \mathbf{p}_2$. Writing out this objective function and taking partial derivatives leads to showing that 

$$\mathbf{X^TXp_2} = \lambda_2 \mathbf{p_2}$$.

From this we learn that:
* The loadings are the eigenvectors of $\mathbf{X^TX}$.
* Sorting the eigenvalues in order from largest to smallest gives the order of the corresponding eigenvectors, the loadings.
* We know from the theory of eigenvalues that if there are distinct eigenvalues, then their eigenvectors are linearly independent (orthogonal).
* We also know the eigenvalues of $\mathbf{X^TX}$ must be real values and positive; this matches with the interpretation that the eigenvalues are proportional to the variance of each score vector.
* Also, the sum of the eigenvalues must add up to sum of the diagonal entries of $\mathbf{X^TX}$, which represents of the total variance of the $\mathbf{X}$ matrix, if all eigenvectors are extracted. So plotting the eigenvalues is equivalent to showing the proportion of variance explained in X by each component. This is not necessarily a good way to judge the number of components to use, but it is a rough guide: use a Pareto plot of the eigenvalues (though in the context of eigenvalue problems, this plot is called a scree plot).

In [8]:
cov = np.cov(X, rowvar = False)
evals , P = LA.eigh(cov)
idx = np.argsort(evals)[::-1]
P = P[:,idx]
evals = evals[idx]
T = np.dot(X, P) 
Sigma=LA.norm(T,axis=0)
print("T (Scores)")
print(T)
print("P (Loadings)")
print(P)
print("Sigma (Variance)")
print(Sigma)

T (Scores)
[[ 4.25324997e+03  8.41288672e-01  8.37858943e-03]
 [-2.97275001e+03  1.25977271e-01 -1.82476780e-01]
 [-3.62475003e+03  1.56843494e-01  1.65224286e-01]
 [ 2.34425007e+03 -1.12410944e+00  8.87390454e-03]]
P (Loadings)
[[-1.21901390e-05 -5.66460727e-01  8.24088736e-01]
 [-9.99999997e-01 -5.32639789e-05 -5.14047691e-05]
 [-7.30130279e-05  8.24088734e-01  5.66460725e-01]]
Sigma (Variance)
[6.74994067e+03 1.41840009e+00 2.46466604e-01]


## Task 1: Test if the loading vectors are orthogonal and orthonormal or not

In [9]:
from itertools import combinations

for i, j in combinations(range(P.shape[1]), 2):
    d = P[:,i].dot(P[:,j])
    print(f"P{i} · P{j} = {d}")

P0 · P1 = -1.8270014701286074e-19
P0 · P2 = -2.7070685671791457e-19
P1 · P2 = -1.0080824159617591e-16


In [10]:
for i in range(P.shape[1]):
    l = np.linalg.norm(P[:,i])
    print(f"||P{i}|| = {l}")

||P0|| = 0.9999999999999999
||P1|| = 1.0
||P2|| = 1.0


The dot products are very small, so the loading vectors are more or less pairwise orthogonal. The norms are close to or exactly unit length. Hence, the loading vectors is approximately orthonormal. 

## Task 2: Test if the scores vectors are orthogonal and orthonormal or not

In [11]:
from itertools import combinations

for i, j in combinations(range(T.shape[1]), 2):
    d = T[:,i].dot(T[:,j])
    print(f"T{i} · T{j} = {d}")

T0 · T1 = -1.8189894035458565e-11
T0 · T2 = -1.830358087318018e-11
T1 · T2 = -1.4030443473700416e-14


In [12]:
for i in range(T.shape[1]):
    l = np.linalg.norm(T[:,i])
    print(f"||T{i}|| = {l}")

||T0|| = 6749.940666380364
||T1|| = 1.41840008831963
||T2|| = 0.24646660404039109


The dot products are very small, so the score vectors are very close to pairwise orthogonal. However, their norms are nowhere near unit length, so the score vectors are not orthonormal. 

## Task 3: Add more columns to the original data matrix by: 
* Make some of the columns to be the linear combination of others
* Duplicate some columns
* Add noise as some columns 
* Add a few columns of categorical values

Then apply PCA to the dataset and report your findings here

In [13]:
# Add linear combination
x = X[:,0] - 2*X[:,1]
X = np.append(X, x[:,None], axis=1)

x = X[:,2] - X[:,0]
X = np.append(X, x[:,None], axis=1)

# Duplicate
x = X[:,1:3]
X = np.append(X, x, axis=1)

# Add noise
x = np.random.uniform(-1, 1, (4,3))
X = np.append(X, x, axis=1)

# Add categorical
x = np.array([[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1]]).T
X = np.append(X, x, axis=1)

print(X.shape)
print(X)


(4, 13)
[[-5.21500000e-01 -4.25325000e+03  3.87500000e-01  8.50597850e+03
   9.09000000e-01 -4.25325000e+03  3.87500000e-01 -3.71249390e-01
   4.00973840e-01  8.70698019e-01  0.00000000e+00  1.00000000e+00
   0.00000000e+00]
 [-1.85500000e-01  2.97275000e+03  2.17500000e-01 -5.94568550e+03
   4.03000000e-01  2.97275000e+03  2.17500000e-01  1.90831421e-01
   1.57254101e-01  6.13245861e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 9.15000000e-02  3.62475000e+03  4.87500000e-01 -7.24940850e+03
   3.96000000e-01  3.62475000e+03  4.87500000e-01  2.48591305e-01
   7.32978128e-02 -1.98154401e-01  1.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 6.15500000e-01 -2.34425000e+03 -1.09250000e+00  4.68911550e+03
  -1.70800000e+00 -2.34425000e+03 -1.09250000e+00 -7.08481855e-01
   6.73947283e-01  8.57206582e-01  0.00000000e+00  0.00000000e+00
   1.00000000e+00]]


In [14]:
U, S, V = np.linalg.svd(X)
P = V.T
T = U @ np.diag(S)

print(f"P (loadings), shape = {P.shape}")
print(P)
print()
print(f"T (scores), shape = {T.shape}")
print(T)

P (loadings), shape = (13, 13)
[[ 4.97457238e-06  2.69491315e-01  5.69687663e-02 -2.31899940e-01
   5.37466765e-01 -3.82501210e-01  1.94088570e-01  1.28411954e-01
  -9.49558583e-02  2.59691783e-01 -4.79036715e-01  8.83386713e-02
  -2.59378825e-01]
 [ 4.08249947e-01  8.98505761e-02  1.89684910e-02 -7.72828796e-02
  -1.61249933e-01 -1.73764244e-01 -6.73061617e-02  1.45209969e-01
  -3.44164566e-01 -6.65705387e-01 -1.77733535e-01 -3.16148738e-01
  -2.09672733e-01]
 [ 2.98102850e-05 -3.94586051e-01 -4.80694293e-02 -1.45031891e-01
  -3.66859239e-01 -3.98920922e-04 -3.61611919e-01 -3.23580039e-02
   2.37476277e-03  3.25049935e-01 -6.29919410e-01 -2.14582731e-01
   7.78044732e-02]
 [-8.16494919e-01  8.97901629e-02  1.90317843e-02 -7.73341809e-02
   2.36643974e-02  3.34742242e-01  4.52241776e-03  9.83685216e-02
  -1.92273745e-01 -2.86681921e-01 -1.81145921e-01 -1.42092857e-01
  -1.56402871e-01]
 [ 2.48357127e-05 -6.64077366e-01 -1.05038196e-01  8.68680487e-02
   6.59465043e-01 -1.32046331e-02 -

In [15]:
from itertools import combinations

for i, j in combinations(range(T.shape[1]), 2):
    d = T[:,i].dot(T[:,j])
    print(f"T{i} · T{j} = {d}")

print()
for i, j in combinations(range(P.shape[1]), 2):
    d = P[:,i].dot(P[:,j])
    print(f"P{i} · P{j} = {d}")

T0 · T1 = 9.094947017729282e-12
T0 · T2 = -2.2737367544323206e-12
T0 · T3 = -9.094947017729282e-13
T1 · T2 = 8.881784197001252e-16
T1 · T3 = 5.551115123125783e-17
T2 · T3 = -2.220446049250313e-16

P0 · P1 = 1.384013559693673e-16
P0 · P2 = 7.216395708389416e-17
P0 · P3 = -8.644985931640519e-17
P0 · P4 = 2.1707457081280605e-16
P0 · P5 = -3.1800102400132374e-16
P0 · P6 = 9.345517369284571e-17
P0 · P7 = 5.4187959304530263e-17
P0 · P8 = 2.16882686219331e-17
P0 · P9 = 2.0352378161700734e-16
P0 · P10 = -1.941238493990129e-16
P0 · P11 = 3.121419890920022e-17
P0 · P12 = -7.230134958062171e-17
P1 · P2 = -1.6943945182504454e-16
P1 · P3 = 3.4677675398328587e-18
P1 · P4 = 3.4365798506155196e-17
P1 · P5 = -7.178535808211429e-17
P1 · P6 = 7.946291903491407e-17
P1 · P7 = 5.2915442496445343e-17
P1 · P8 = -8.693792135521295e-17
P1 · P9 = -2.229740455894779e-16
P1 · P10 = -7.463562591587836e-17
P1 · P11 = -7.420054514741911e-17
P1 · P12 = -1.6774842002576154e-16
P2 · P3 = -5.264798818083943e-17
P2 · P4 =

In [16]:
for i in range(T.shape[1]):
    l = np.linalg.norm(T[:,i])
    print(f"||T{i}|| = {l}")

print()
for i in range(P.shape[1]):
    l = np.linalg.norm(P[:,i])
    print(f"||P{i}|| = {l}")

||T0|| = 16533.84328925055
||T1|| = 2.959088375039947
||T2|| = 1.5537693375924309
||T3|| = 0.8948286799484996

||P0|| = 1.0000000000000002
||P1|| = 1.0
||P2|| = 0.9999999999999999
||P3|| = 1.0
||P4|| = 1.0
||P5|| = 0.9999999999999999
||P6|| = 1.0
||P7|| = 1.0
||P8|| = 0.9999999999999999
||P9|| = 0.9999999999999999
||P10|| = 1.0
||P11|| = 1.0
||P12|| = 1.0


In [17]:
explained_variance = S/np.sum(S)
for i, ev in enumerate(explained_variance):
    print(f"Explained variance by PC-{i}: {100*ev:.5f}%")

Explained variance by PC-0: 99.96730%
Explained variance by PC-1: 0.01789%
Explained variance by PC-2: 0.00939%
Explained variance by PC-3: 0.00541%


All loadings and score vectors are pairwise orthogonal as they should be, and all the loadings are also unit length. This is what is expected from the PCA, and we get this even though we have appended all different types of data. 

We also see that almost all of the explained variance lies in the first PC. We also see that the magnitude of the first scores vector is significantly larger than the others. Perhaps it is that the scores vector explaining the largest variance is the one with largest norm?

